# pandasToMatrix
Authors: Ardel Alegre, Brian Wang
### Sumarry
pandas to matrix accepts a date (##-##-20##), starting hour, total number of hours, and multiple source indicator as its parameters. pandasToMatrix will take the dataframes given these parameters and return a 13 dimensional matrix that represents the dataframes.

In [1]:
import pandas as pd
import numpy as np
import copy

In [44]:
def source_matching_single(dfs):
    
    df = dfs.drop_duplicates(subset=['Time In Seconds', 'Microphone Number'])
    
    master_list = []
    start_time = df['Time In Seconds'][0]
    split = start_time + 0.0077
    

    end_time = df['Time In Seconds'].iloc[-1]
    temp = df.iloc[0]
    temp['X'] = None
    temp['Y'] = None
    temp['Z'] = None
    temp['Microphone Number'] = None
    array_0 = temp 
    array_1 = temp
    array_2 = temp
    array_3 = temp
    counter = 0
    #Assuming only 1 channel for each array
    
    for row in df.iterrows():
        if row[1]['Time In Seconds'] >= split:
            data_point = [None] * 13
            data_point[0] = array_0['X']
            data_point[1] = array_0['Y']
            data_point[2] = array_0['Z']
            data_point[3] = array_1['X']
            data_point[4] = array_1['Y']
            data_point[5] = array_1['Z']
            data_point[6] = array_2['X']
            data_point[7] = array_2['Y']
            data_point[8] = array_2['Z']
            data_point[9] = array_3['X']
            data_point[10] = array_3['Y']
            data_point[11] = array_3['Z']
            data_point[12] = split - .0077
            
            if data_point[0] != None  or data_point[3] != None or  data_point[6] != None or data_point[9] != None:
                master_list.append(data_point)

            filler = copy.copy(row[1])
            filler['X'] = None
            filler['Y'] = None
            filler['Z'] = None
            array_0 = filler
            array_1 = filler
            array_2 = filler
            array_3 = filler
            
            split = row[1]['Time In Seconds'] + 0.0077

            
                
        if row[1]['Microphone Number'] == 0:
            array_0 = row[1]
            
        elif row[1]['Microphone Number'] == 1:
            array_1 = row[1]
            
        elif row[1]['Microphone Number'] == 2:
            array_2 = row[1]

        else:
            array_3 = row[1]
    
    return master_list

In [15]:
def main(date, starting_hour=0, num_hours=24, multiple_sources=False): 
    matricies = []
    ending_hour = starting_hour + num_hours
    if(ending_hour >= 25):
        print("Hours must end before end of day.")
        return
    
    hours = []
    for i in range(starting_hour, ending_hour):
        if(i > 9):
            hours.append(str(i))
        else:
            hours.append('0' + str(i))
    for hour in hours:
        df = read_csv('/home/ardelalegre/google-drive/ODAS/dataframes/combined/date/' + hour + '.csv')
        if(multiple_sources):
            pass
        else:
            temp = convert_single(df)
        try:
            matricies = matricies.append(temp)
        except:
            print("Too many hours")
    matrix = np.asarray(matricies)
    return matrix
